In [ ]:
# import packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk  
import tensorflow as tf
!pip install transformers
import torch

# BERT and T5 are easily imported from Hugging Face's transformer library (https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel
from transformers import BertTokenizer, TFBertModel, T5Tokenizer,  MT5Tokenizer, TFT5ForConditionalGeneration
from torch.utils.data import TensorDataset, random_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import class_weight 


# Keras functional API
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import Input

import math

!pip install pyspellchecker  
from spellchecker import SpellChecker
  
import random
from random import randint
import more_itertools
from nltk import tokenize
nltk.download('punkt')

tf.test.gpu_device_name()



     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 895 kB 34.8 MB/s 
     |████████████████████████████████| 3.3 MB 70.4 MB/s 
     |████████████████████████████████| 636 kB 82.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 2.7 MB 7.6 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'/device:GPU:0'

# Load BERT and Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
# import transformers
#!pip install -q transformers

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# Create a dataframe for adding prompt of each essay set to get the prompt-relevant score
prompt = pd.DataFrame({"essay_set":[1,2,3,4,5,6,7,8],
                       "prompt":["More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
                                "Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.",
                                "Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
                                "Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.",
                                "Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
                                "Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.",
                                "Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.",
                                "We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part."]})
prompt

,essay_set,prompt
0,1,"More and more people use computers, but not ev..."
1,2,Write a persuasive essay to a newspaper reflec...
2,3,Write a response that explains how the feature...
3,4,Write a response that explains why the author ...
4,5,Describe the mood created by the author in the...
5,6,"Based on the excerpt, describe the obstacles t..."
6,7,Write about patience. Being patient means that...
7,8,We all understand the benefits of laughter. Fo...


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test_set.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')

# Merge training and prompt data

In [ ]:
dev

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,NaN
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,NaN
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,NaN
...,...,...,...,...,...
4213,21933,8,Have you ever noticed that if two little kids...,24933,NaN
4214,21934,8,Laughter @CAPS1 I ...,24934,NaN
4215,21935,8,Laughter in @CAPS1 A laugh is not just an act...,24935,NaN
4216,21937,8,LAUGHTER @CAPS1 i was younger my friend live...,24937,NaN


# Data Processing

In [ ]:
def clean_anonymization(essay):
  res=[]
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [ ]:
train1=train.merge(prompt,on='essay_set',how='left')
train1.head(3)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,prompt
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev..."
1,2,1,Dear I believe that using computers will benef...,5,4,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev..."
2,3,1,"Dear, More and more people use computers, but ...",4,3,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev..."


In [ ]:
def split_list(a_list):
    tenth = len(a_list)//10
    return a_list[:tenth], a_list[tenth:]

def normalize(df):
    df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')
    return df

def tokenized(df, examples):
    total_example_iter = random.sample(range(0, len(df)-1), examples)
    example_adv, example_iter = split_list(total_example_iter)
    input_ids = []
    token_type_ids = []
    attention_mask = []
    gold_scores = []
    accounting = {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0}
    for iteration in total_example_iter:
      if iteration in example_iter:
        z_prompt = df['prompt'].iloc[iteration]
        essay = df['essay'].iloc[iteration]
        encoded_dict = tokenizer(essay, z_prompt, 
                  max_length=512,
                  padding='max_length',
                  truncation = True, 
                  return_tensors='tf')
        input_ids.append(encoded_dict.input_ids)
        token_type_ids.append(encoded_dict.token_type_ids)
        attention_mask.append(encoded_dict.attention_mask)
        gold_scores.append(df['normalized_score'].iloc[iteration])
      else:
        essay = df['essay'].iloc[iteration]
        p_essay_set = df['essay_set'].iloc[iteration]
        accounting[str(p_essay_set)] -= 1
        pos_accounting = [k for k,v in accounting.items() if v < 0]
        if len(pos_accounting) == 0:
          r_essay_set = random.randint(1, 8)
          while r_essay_set == p_essay_set:
            r_essay_set = random.randint(1, 8)
        else:
          r_essay_set = pos_accounting[0]
        accounting[str(r_essay_set)] += 1
        z_prompt = prompt.to_dict()['prompt'][int(r_essay_set)-1]
        encoded_dict = tokenizer(essay, z_prompt, 
                  max_length=512,
                  padding='max_length',
                  truncation = True, 
                  return_tensors='tf')
        input_ids.append(encoded_dict.input_ids)
        token_type_ids.append(encoded_dict.token_type_ids)
        attention_mask.append(encoded_dict.attention_mask)
        gold_scores.append(0)
    
    return {'gold_scores': gold_scores,'input_ids': tf.concat([z for z in input_ids], axis=0), 'token_type_ids': tf.concat([z for z in token_type_ids], axis=0), 'attention_mask': tf.concat([z for z in attention_mask], axis=0)}


def tokenized_test(df, examples):
    example_iter = random.sample(range(0, len(df)), examples)
    input_ids = []
    token_type_ids = []
    attention_mask = []
    for iteration in example_iter:
      prompt = df['prompt'].iloc[iteration]
      essay = df['essay'].iloc[iteration]
      encoded_dict = tokenizer(essay, prompt, 
                max_length=512,
                padding='max_length',
                truncation = True, 
                return_tensors='tf')

      input_ids.append(encoded_dict.input_ids)
      token_type_ids.append(encoded_dict.token_type_ids)
      attention_mask.append(encoded_dict.attention_mask)
    
    return {'input_ids': tf.concat([z for z in input_ids], axis=0), 'token_type_ids': tf.concat([z for z in token_type_ids], axis=0), 'attention_mask': tf.concat([z for z in attention_mask], axis=0)}


In [ ]:
def first_stage_model(hidden_size = 300, train_layers = -1, learning_rate= 3e-5, dropout_value=0.5):

    optimizer=tf.keras.optimizers.Adam(learning_rate)   

    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}
    
    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)

    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])

    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(classification_token)

    dropout = tf.keras.layers.Dropout(dropout_value)(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(dropout)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss="mse",
                            metrics=["mae"])


    return classification_model

In [ ]:
train1 = normalize(train1)
train1['sent'] = train1['essay'].apply(lambda x: tokenize.sent_tokenize(x))
y = train1['normalized_score']


X_train, X_test, y_train, y_test = train_test_split(train1, y, test_size=0.33, random_state=0)
enc_train = tokenized(X_train, len(X_train) - 1)
enc_val = tokenized(X_test, len(X_test) - 1)
class_weights = class_weight.compute_class_weight('balanced',pd.Series(enc_train['gold_scores']).value_counts().index,pd.Series(enc_train['gold_scores']))
class_weights = dict(enumerate(class_weights))
fsm = first_stage_model()
fsm.fit([enc_train['input_ids'], enc_train['token_type_ids'], enc_train['attention_mask']], pd.Series(enc_train['gold_scores']), validation_data = ([enc_val['input_ids'], enc_val['token_type_ids'], enc_val['attention_mask']], pd.Series(enc_val['gold_scores'])) , epochs=1, batch_size=10, class_weight=class_weights, shuffle=True)


870/870 [==============================] - 731s 823ms/step - loss: 0.0069 - mae: 0.1764 - val_loss: 0.0557 - val_mae: 0.1488


In [ ]:
enc_test = tokenized_test(adv, len(adv))

a = fsm.predict([enc_test['input_ids'], enc_test['token_type_ids'], enc_test['attention_mask']])

#Hyperparameter Tuning

In [ ]:
train1 = normalize(train1)
y = train1['normalized_score']

X_train, X_test, y_train, y_test = train_test_split(train1, y, test_size=0.33, random_state=0)
enc_train = tokenized(X_train, 1500)
enc_val = tokenized(X_test, 1500)
class_weights = class_weight.compute_class_weight('balanced',pd.Series(enc_train['gold_scores']).value_counts().index,pd.Series(enc_train['gold_scores']))
class_weights = dict(enumerate(class_weights))

dropout_values = [0.3,0.4,0.5]
hidden_sizes = [100,300, 500]
batch_sizes = [10]
learning_rates = [1e-5, 3e-5]
histories = {}
i = 0
for dropout_value in dropout_values:
  for hidden_size in hidden_sizes:
    for batch_size in batch_sizes:
      for learning_rate in learning_rates:
        print('\n' + 'iteration ' + str(i) + ' dropout_value: ' + str(dropout_value) + ' hidden_size: ' + str(hidden_size) + ' batch_size: ' + str(batch_size) + ' learning_rate ' + str(learning_rate) + "\n")
        fsm = first_stage_model(hidden_size=hidden_size, dropout_value=dropout_value, learning_rate=learning_rate)
        history = fsm.fit([enc_train['input_ids'], enc_train['token_type_ids'], enc_train['attention_mask']], pd.Series(enc_train['gold_scores']), validation_data = ([enc_val['input_ids'], enc_val['token_type_ids'], enc_val['attention_mask']], pd.Series(enc_val['gold_scores'])) , epochs=2, batch_size=batch_size, class_weight=class_weights, shuffle=True, verbose=2)
        histories[(dropout_value, hidden_size, batch_size, learning_rate)] = history.history['val_loss']
        i += 1


iteration 0 dropout_value: 0.3 hidden_size: 100 batch_size: 10 learning_rate 1e-05

Epoch 1/2
150/150 - 146s - loss: 0.0059 - mae: 0.1585 - val_loss: 0.0392 - val_mae: 0.1534
Epoch 2/2
150/150 - 136s - loss: 0.0036 - mae: 0.1230 - val_loss: 0.0283 - val_mae: 0.1295

iteration 1 dropout_value: 0.3 hidden_size: 100 batch_size: 10 learning_rate 3e-05

Epoch 1/2
150/150 - 147s - loss: 0.0046 - mae: 0.1365 - val_loss: 0.0299 - val_mae: 0.1301
Epoch 2/2
150/150 - 136s - loss: 0.0028 - mae: 0.1064 - val_loss: 0.0206 - val_mae: 0.0995

iteration 2 dropout_value: 0.3 hidden_size: 300 batch_size: 10 learning_rate 1e-05

Epoch 1/2
150/150 - 146s - loss: 0.0036 - mae: 0.1196 - val_loss: 0.0226 - val_mae: 0.1080
Epoch 2/2
150/150 - 136s - loss: 0.0026 - mae: 0.1012 - val_loss: 0.0247 - val_mae: 0.1147

iteration 3 dropout_value: 0.3 hidden_size: 300 batch_size: 10 learning_rate 3e-05

Epoch 1/2
150/150 - 147s - loss: 0.0033 - mae: 0.1129 - val_loss: 0.0221 - val_mae: 0.1052
Epoch 2/2
150/150 - 136